## Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

2024-08-26 09:20:27.279591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prepare paths of input images and target segmentation masks

In [2]:
input_dir = "images/"
target_dir = "annotations/trimaps/"
img_size = (160, 160)
num_classes = 3
batch_size = 32

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)

Number of samples: 7390
images/Abyssinian_1.jpg | annotations/trimaps/Abyssinian_1.png
images/Abyssinian_10.jpg | annotations/trimaps/Abyssinian_10.png
images/Abyssinian_100.jpg | annotations/trimaps/Abyssinian_100.png
images/Abyssinian_101.jpg | annotations/trimaps/Abyssinian_101.png
images/Abyssinian_102.jpg | annotations/trimaps/Abyssinian_102.png
images/Abyssinian_103.jpg | annotations/trimaps/Abyssinian_103.png
images/Abyssinian_104.jpg | annotations/trimaps/Abyssinian_104.png
images/Abyssinian_105.jpg | annotations/trimaps/Abyssinian_105.png
images/Abyssinian_106.jpg | annotations/trimaps/Abyssinian_106.png
images/Abyssinian_107.jpg | annotations/trimaps/Abyssinian_107.png


## Load data paths

In [3]:
import pickle

# Load the list from the file using pickle
def load_paths_file(file_name):
    with open(f"./paths/{file_name}.pkl", "rb") as file:
        paths = pickle.load(file)
    return paths

train_input_img_paths = load_paths_file("train_input_img_paths")
train_target_img_paths = load_paths_file("train_target_img_paths")
val_input_img_paths = load_paths_file("val_input_img_paths")
val_target_img_paths = load_paths_file("val_target_img_paths")
test_input_img_paths = load_paths_file("test_input_img_paths")
test_target_img_paths = load_paths_file("test_target_img_paths")

print("Data paths loaded")

print(train_input_img_paths)

Data paths loaded
['images/samoyed_64.jpg', 'images/yorkshire_terrier_21.jpg', 'images/shiba_inu_143.jpg', 'images/pomeranian_86.jpg', 'images/boxer_85.jpg', 'images/Birman_17.jpg', 'images/beagle_64.jpg', 'images/german_shorthaired_182.jpg', 'images/staffordshire_bull_terrier_194.jpg', 'images/Egyptian_Mau_218.jpg', 'images/wheaten_terrier_92.jpg', 'images/basset_hound_115.jpg', 'images/chihuahua_152.jpg', 'images/Siamese_18.jpg', 'images/beagle_196.jpg', 'images/Bengal_120.jpg', 'images/newfoundland_6.jpg', 'images/english_cocker_spaniel_95.jpg', 'images/Siamese_34.jpg', 'images/havanese_7.jpg', 'images/samoyed_174.jpg', 'images/american_bulldog_59.jpg', 'images/english_cocker_spaniel_139.jpg', 'images/havanese_54.jpg', 'images/chihuahua_191.jpg', 'images/saint_bernard_197.jpg', 'images/japanese_chin_110.jpg', 'images/pug_169.jpg', 'images/basset_hound_22.jpg', 'images/British_Shorthair_92.jpg', 'images/Siamese_179.jpg', 'images/miniature_pinscher_24.jpg', 'images/german_shorthaired_

## Load datasets

In [4]:
train_dataset_path = 'saved_data/train_dataset'
valid_dataset_path = 'saved_data/valid_dataset'
test_dataset_path = 'saved_data/test_dataset'

# Load datasets
train_dataset = tf.data.Dataset.load(train_dataset_path)
valid_dataset = tf.data.Dataset.load(valid_dataset_path)
test_dataset = tf.data.Dataset.load(test_dataset_path)

print("Datasets loaded.")

Datasets loaded.


2024-08-26 09:20:31.631394: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Function to save results

In [5]:
saved_model_dir = "./saved_models"

# Function to extract image name for the image path 
def extract_image_name(image_path):
    return os.path.basename(image_path)

# Prepare lists to hold all results
individual_results = []
average_results = []

# Helper function to save individual and average results
def save_results(platform, model_name, epochs, num_filters, norm_status, quantization, iteration, execution_times, peak_lists, iou_scores, dice_scores):
    for i, (iou, dice) in enumerate(zip(iou_scores, dice_scores)):
        individual_results.append({
            "Iteration": iteration + 1,
            "Platform": platform,
            "Model": model_name,
            "Epoch": epochs,
            "Filters": num_filters,
            "Normalization": norm_status,
            "Quantization": quantization,
            "Image_Name": extract_image_name(test_target_img_paths[i]),
            "Execution_time": execution_times[i],
            "Memory_usage": peak_lists[i],
            "IoU_score": iou,
            "Dice_score": dice
        })

    average_results.append({
        "Model": model_name,
        "Epoch": epochs,
        "Filters": num_filters,
        "Normalization": norm_status,
        "Quantization": quantization,
        "Average_Execution_time": np.mean(execution_times),
        "Average_Memory_usage": np.mean(peak_lists),
        "Average_IoU_score": np.mean(iou_scores),
        "Average_Dice_score": np.mean(dice_scores)
    })
    print(f"Completed: {model_name} | Iteration: {iteration + 1}")    

## Calculate Dice and IoU scores

In [6]:
# Function to calculate Dice coefficient
def dice_coefficient(y_true, y_pred):
    y_true = tf.keras.backend.cast(y_true, 'float32')
    y_pred = tf.keras.backend.cast(y_pred, 'float32')
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true * y_pred)
    return (2. * intersection + 1) / (tf.keras.backend.sum(y_true) + tf.keras.backend.sum(y_pred) + 1)

# Function to calculate IoU score
def iou_score(y_true, y_pred):
    y_true = tf.keras.backend.cast(y_true, 'float32')
    y_pred = tf.keras.backend.cast(y_pred, 'float32')
    y_true = tf.keras.backend.flatten(y_true)
    y_pred = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true * y_pred)
    union = tf.keras.backend.sum(y_true) + tf.keras.backend.sum(y_pred) - intersection
    return (intersection + 1) / (union + 1)

In [7]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to convert grayscale mask to binary mask
def get_binary_mask(mask, threshold = 1):
    """Convert grayscale mask to binary mask with black background and white object."""
    return (mask > threshold).astype(np.uint8)

# Helper function to calculate metrics for predictions
def calculate_metrics(test_preds_array):
    iou_scores, dice_scores = [],[]
    for i, target_img_path in enumerate(test_target_img_paths):
        # Load and normalize the ground truth mask
        ground_truth_mask = img_to_array(load_img(target_img_path, color_mode="grayscale", target_size=img_size))
        ground_truth_mask = np.squeeze(ground_truth_mask / np.max(ground_truth_mask)).astype(np.uint8)
        
        # Get the model's prediction, resize, and convert to binary
        pred_mask = tf.image.resize(test_preds_array[i], img_size).numpy()
        pred_mask = get_binary_mask(np.argmax(pred_mask, axis=-1))
        
        # Append scores to the lists
        iou_scores.append(iou_score(ground_truth_mask, pred_mask).numpy())
        dice_scores.append(dice_coefficient(ground_truth_mask, pred_mask).numpy())
    return iou_scores, dice_scores

## Processing the Tensorflow model

In [8]:
import time
import tracemalloc

# Helper function to process a single model configuration
def process_model(epochs, num_filters, normalization, quantization):
    '''
    This function helps to:
    # Predict on the test dataset 
    # Caculate the execution time and memory usage for each image prediction
    # Perform Dice and IoU calculations
    # Save the model
    '''     
    
    norm_status = "with_batch_norm" if normalization else "without_batch_norm"
    model_name = f"model_epochs_{epochs}_filters_{num_filters}_{norm_status}"
    model_path = os.path.join(saved_model_dir, model_name)

    # Load the model
    model = tf.saved_model.load(model_path)
    infer = model.signatures['serving_default']
    print(f"Model {model_name} loaded")
    
    # Initialize lists for scores and metrics
    test_preds, execution_times, peak_lists = [], [], []

    # Loop through each batch and image
    for batch in test_dataset:
        # Process each image in the batch
        for img in batch[0]:
            start_time = time.time()
            tracemalloc.start()

            # Prepare input data and perform inference
            input_tensor = tf.expand_dims(img, axis=0)
           
            preds = infer(input_tensor)
            test_preds.append(preds)

            # Record time and memory usage
            execution_times.append(time.time() - start_time)
            
            _, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            peak_lists.append(peak / 1024)  # Convert to KB
            
    output_key = 'output_0'
    
    # Combine predictions and calculate metrics
    if test_preds:
        final_test_preds = {key: np.concatenate([pred[key].numpy() for pred in test_preds], axis=0) for key in test_preds[0]}
        test_preds_array = final_test_preds[output_key]
        iou_scores, dice_scores = calculate_metrics(test_preds_array)
    else:
        iou_scores, dice_scores = [], []

    return model_name, execution_times, peak_lists, iou_scores, dice_scores    

## Processing the Tensorflow Lite model

In [9]:
# Helper function to process a single TFLite model configuration
def process_tflite_model(epochs, num_filters, normalization, quantization):
    '''
    This function helps to:
    # Predict on the test dataset 
    # Caculate the execution time and memory usage for each image prediction
    # Perform Dice and IoU calculations
    # Save model
    '''
    norm_status = "with_batch_norm" if normalization else "without_batch_norm"
    model_name = f"model_epochs_{epochs}_filters_{num_filters}_{norm_status}_{quantization}.tflite"
    model_path = os.path.join(saved_model_dir, model_name)

    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path)
    interpreter.allocate_tensors()  

    # Get input and output details
    input_index = interpreter.get_input_details()[0]['index']
    output_index = interpreter.get_output_details()[0]['index']
    print(f"Model {model_name} loaded")
    
    # Initialize lists for scores and metrics
    test_preds, execution_times, peak_lists = [], [], []

    # Loop through each batch and image
    for batch in test_dataset:
        for img in batch[0]:
            start_time = time.time()
            tracemalloc.start()
            
            # Prepare input data and run inference
            interpreter.set_tensor(input_index, np.expand_dims(img.numpy(), axis=0))
            interpreter.invoke()
            
            # Extract and store the prediction
            test_preds.append(interpreter.get_tensor(output_index))
            # Record time and memory usage
            execution_times.append(time.time() - start_time)
            
            _, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            peak_lists.append(peak / 1024)  # Convert to KB
        
    # Convert the list of predictions to a single numpy array
    test_preds_array = np.vstack(test_preds)
    iou_scores, dice_scores = calculate_metrics(test_preds_array)
    
    return model_name, execution_times, peak_lists, iou_scores, dice_scores

## Main Function

In [12]:
import pandas as pd

# Experiment parameters
epochs_list = [50, 100]
num_filters_list = [32, 64, 128]
quantization_types = ['no', 'default', 'float16']
iterations = 10
platform = "CPU"
quantization = "Default"
output_key = 'output_0'
saved_model_dir = "./saved_models"

# Main loop to process all configurations
for epochs in epochs_list:
    for num_filters in num_filters_list:        
        for normalization in [True, False]:
            for quantization in quantization_types: 
                for iteration in range(iterations):
                    model_processing_function = process_model if quantization == 'no' else process_tflite_model

                    # Process the model and get the results
                    model_name, execution_times, peak_lists, iou_scores, dice_scores = model_processing_function(
                        epochs, num_filters, normalization, quantization)

                    # Save the results
                    save_results(platform, model_name, epochs, num_filters, normalization, quantization, iteration, 
                                execution_times, peak_lists, iou_scores, dice_scores)

# Save results to CSV files
pd.DataFrame(individual_results).to_csv("model_tflite_performance_individual_results.csv", index=False)
pd.DataFrame(average_results).to_csv("model_tflite_performance_average_results.csv", index=False)

print("Results saved to CSV files")

Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 1
Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 2
Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 3
Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 4
Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 5
Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 6
Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 7
Model model_epochs_50_filters_32_with_batch_norm loaded
Completed: model_epochs_50_filters_32_with_batch_norm | Iteration: 8


INFO: Created TensorFlow Lite delegate for select TF ops.
2024-08-26 13:11:16.898190: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
INFO: TfLiteFlexDelegate delegate: 3 nodes delegated out of 176 nodes with 3 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Completed: model_epochs_50_filters_32_with_batch_norm_default.tflite | Iteration: 1
Model model_epochs_50_filters_32_with_batch_norm_default.tflite loaded


2024-08-26 13:12:24.194990: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Completed: model_epochs_50_filters_32_with_batch_norm_default.tflite | Iteration: 2
Model model_epochs_50_filters_32_with_batch_norm_default.tflite loaded


...

Completed: model_epochs_50_filters_32_without_batch_norm_float16.tflite | Iteration: 5
Model model_epochs_50_filters_32_without_batch_norm_float16.tflite loaded


2024-08-26 13:59:51.564730: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Completed: model_epochs_50_filters_32_without_batch_norm_float16.tflite | Iteration: 6
Model model_epochs_50_filters_32_without_batch_norm_float16.tflite loaded


2024-08-26 14:00:53.879517: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


KeyboardInterrupt: 